#### Data was released Mar 29. Before summer uptick.
#### SQL query below to extract and format data correctly

`SELECT DISTINCT l.zip, l.county, p.person_id, p.gender_source_value, p.birth_datetime, p.death_datetime, p.race_source_value, p.ethnicity_source_value, c.condition_start_datetime, c.condition_concept_id
FROM person p
INNER JOIN condition_occurrence c ON c.person_id = p.person_id
INNER JOIN location l on l.location_id = p.location_id
WHERE condition_concept_id = '37311061' --Disease caused by 2019-nCoV`


In [44]:
import pandas as pd

#data contains positive cases, gender, race/ethnicity, death, birth
data = pd.read_csv ('data-1605136079581.csv')
pop = pd.read_csv ('population2010.csv')

In [49]:
#remove the word 'county' from the county column
data['county'] = data['county'].str.split(' ').str[0]
data['condition_month'] = pd.DatetimeIndex(data['condition_start_datetime']).month
death = data [data['death_datetime'].notna()]
data

,zip,county,person_id,gender_source_value,birth_datetime,death_datetime,race_source_value,ethnicity_source_value,condition_start_datetime,condition_concept_id,condition_month
0,NaN,Worcester,8573,F,1959-06-24 00:00:00,NaN,black,hispanic,2020-02-20 00:00:00,37311061,2
1,1810.0,Essex,3432,F,1966-04-01 00:00:00,2020-03-03 00:00:00,black,nonhispanic,2020-02-04 00:00:00,37311061,2
2,2301.0,Plymouth,8285,M,1948-10-18 00:00:00,NaN,white,nonhispanic,2020-02-14 00:00:00,37311061,2
3,2457.0,Norfolk,8741,F,1991-08-28 00:00:00,2020-02-17 00:00:00,white,nonhispanic,2020-01-27 00:00:00,37311061,1
4,2110.0,Suffolk,7577,M,2002-12-31 00:00:00,NaN,white,nonhispanic,2020-03-15 00:00:00,37311061,3
...,...,...,...,...,...,...,...,...,...,...,...
1900,1983.0,Essex,10536,F,1976-07-25 00:00:00,NaN,white,nonhispanic,2020-03-22 00:00:00,37311061,3
1901,NaN,Hampden,11130,M,1973-10-04 00:00:00,NaN,white,nonhispanic,2020-03-05 00:00:00,37311061,3
1902,NaN,Worcester,3746,F,2002-12-21 00:00:00,NaN,black,nonhispanic,2020-01-07 00:00:00,37311061,1
1903,2121.0,Suffolk,7880,F,1951-01-13 00:00:00,2020-02-15 00:00:00,asian,nonhispanic,2020-02-10 00:00:00,37311061,2


In [50]:
#cases by county
#some persons have multiple entries (unique cases)
data_pos = data.groupby(['county', 'condition_month','gender_source_value','race_source_value', 'ethnicity_source_value']).size().reset_index(name='positive_counts')

data_death = death.groupby(['county', 'condition_month','gender_source_value','race_source_value', 'ethnicity_source_value']).size().reset_index(name='death_counts')


In [59]:
data_flat = pd.merge (data_pos, data_death, on = ['county', 'condition_month', 'gender_source_value', 'race_source_value','ethnicity_source_value'], how="outer")

In [60]:
data_flat_pop_fips = pd.merge (data_flat, pop, on = ['county'])
data_flat_pop_fips

,county,condition_month,gender_source_value,race_source_value,ethnicity_source_value,positive_counts,death_counts,fips_code,population_2010
0,Barnstable,1,F,white,nonhispanic,9,1.0,25001,215888
1,Barnstable,1,M,white,nonhispanic,9,6.0,25001,215888
2,Barnstable,2,F,white,hispanic,1,NaN,25001,215888
3,Barnstable,2,F,white,nonhispanic,4,NaN,25001,215888
4,Barnstable,2,M,black,nonhispanic,2,1.0,25001,215888
...,...,...,...,...,...,...,...,...,...
235,Worcester,3,F,white,nonhispanic,27,2.0,25027,798552
236,Worcester,3,M,asian,nonhispanic,1,NaN,25027,798552
237,Worcester,3,M,black,hispanic,1,NaN,25027,798552
238,Worcester,3,M,white,hispanic,2,NaN,25027,798552


In [62]:
data_flat_pop_fips.to_csv('covid_ma_data.csv', sep = ',')